In [ ]:
import pandas as pd
import numpy as np
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("data2.csv")

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [ ]:
print(df.isnull().sum())

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
Sales          0
dtype: int64


In [ ]:
# Temizleme fonksiyonu
def clean_description(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Noktalama ve sayıları kaldır
    words = text.split()
    words = [word for word in words if word not in stopwords.words('english')]  # Stopwords'leri çıkar
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]  # Köklerine indirgeme
    return ' '.join(words)

df['Cleaned_Description'] = df['Description'].apply(clean_description)

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales,Cleaned_Description
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,white hanging heart tlight holder
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,white metal lantern
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,cream cupid heart coat hanger
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,knitted union flag hot water bottle
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,red woolly hottie white heart


In [ ]:
df_cleaned = df[~df["InvoiceNo"].astype(str).str.startswith("C")]


In [ ]:
print(df_cleaned)

       InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
406824    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
406825    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
406826    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
406827    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
406828    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

                InvoiceDate  UnitPrice  CustomerID         Country  Sales  

In [ ]:
from gensim.models import Word2Vec
from sklearn.cluster import DBSCAN
import numpy as np

# Ürün açıklamalarını kelime listesine dönüştür
sentences = [desc.split() for desc in df_cleaned['Cleaned_Description']]

# tüm açıklamaların kelime listelerinden oluşan bir liste listesi (nested list) elde edilir.

In [ ]:
# Word2Vec modeli eğit
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)


In [ ]:
# Her açıklamanın vektörünü al (kelime vektörlerinin ortalaması)
def vectorize_text(text):
    words = text.split()
    vectors = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(100)

df_cleaned['Vector'] = df_cleaned['Cleaned_Description'].apply(vectorize_text)


C:\Users\EXCALIBUR\AppData\Local\Temp\ipykernel_17000\2118305219.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Vector'] = df_cleaned['Cleaned_Description'].apply(vectorize_text)


In [ ]:
# DBSCAN kümeleme
dbscan = DBSCAN(eps=0.5, min_samples=5, metric='euclidean')
df_cleaned['Category'] = dbscan.fit_predict(np.vstack(df_cleaned['Vector']))


C:\Users\EXCALIBUR\AppData\Local\Temp\ipykernel_17000\217773369.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Category'] = dbscan.fit_predict(np.vstack(df_cleaned['Vector']))


In [ ]:
# Kümeleme sonuçlarını kontrol et
print(df_cleaned[['StockCode', 'Description', 'Category']])


       StockCode                          Description  Category
0         85123A   WHITE HANGING HEART T-LIGHT HOLDER         0
1          71053                  WHITE METAL LANTERN         1
2         84406B       CREAM CUPID HEARTS COAT HANGER         2
3         84029G  KNITTED UNION FLAG HOT WATER BOTTLE         3
4         84029E       RED WOOLLY HOTTIE WHITE HEART.         4
...          ...                                  ...       ...
406824     22613          PACK OF 20 SPACEBOY NAPKINS      1715
406825     22899         CHILDREN'S APRON DOLLY GIRL        402
406826     23254        CHILDRENS CUTLERY DOLLY GIRL       2514
406827     23255      CHILDRENS CUTLERY CIRCUS PARADE      2516
406828     22138        BAKING SET 9 PIECE RETROSPOT       2159

[397924 rows x 3 columns]


In [ ]:
# Kategorilere göre veri gruplama
categories = df_cleaned.groupby('Category')['Description'].apply(lambda x: ' '.join(x)).reset_index()

# TF-IDF veya CountVectorizer kullanarak her kategorinin en sık geçen kelimeleri
from sklearn.feature_extraction.text import TfidfVectorizer

# Kategori bazında kelime frekansı hesaplama
tfidf = TfidfVectorizer(stop_words='english', max_features=10)
for i, category in categories.iterrows():
    category_text = category['Description']
    tfidf_matrix = tfidf.fit_transform([category_text])
    feature_names = tfidf.get_feature_names_out()
    print(f"Kategori {category['Category']}: {', '.join(feature_names)}")


Kategori -1: black, blue, bracelet, earrings, flower, glass, heart, necklace, pink, vintage
Kategori 0: hanging, heart, holder, light, white
Kategori 1: lantern, metal, white
Kategori 2: coat, cream, cupid, hanger, hearts
Kategori 3: bottle, flag, hot, knitted, union, water
Kategori 4: heart, hottie, red, white, woolly
Kategori 5: babushka, boxes, nesting, set
Kategori 6: frosted, glass, holder, light, star
Kategori 7: hand, jack, union, warmer
Kategori 8: dot, hand, polka, red, warmer
Kategori 9: assorted, bird, colour, ornament
Kategori 10: bedroom, livingroom, playhouse, poppy
Kategori 11: kitchen, playhouse, poppy
Kategori 12: charlotte, doll, feltcraft, princess
Kategori 13: cosy, ivory, knitted, mug
Kategori 14: assorted, box, colour, teaspoons
Kategori 15: blocks, box, jigsaw, vintage
Kategori 16: alphabet, blocks, box, vintage
Kategori 17: block, building, home, word
Kategori 18: block, building, love, word
Kategori 19: doormat, england, new
Kategori 20: jam, jars, making, set


In [ ]:
# Kategorileri sayısal olarak kısıtlama (örneğin 20 kategori yapmak)
from sklearn.cluster import KMeans

# DBSCAN ile oluşturulan kategorileri KMeans ile yeniden düzenleyebilirsin
kmeans = KMeans(n_clusters=30, random_state=42)
df_cleaned['Category'] = kmeans.fit_predict(np.vstack(df_cleaned['Vector']))


C:\Users\EXCALIBUR\AppData\Local\Temp\ipykernel_17000\2818240951.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Category'] = kmeans.fit_predict(np.vstack(df_cleaned['Vector']))


In [ ]:
import numpy as np

# TF-IDF vektörleri veya Word2Vec vektörleri ile çalışan model
# Burada TF-IDF örneği gösteriliyor

from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF modelini oluştur
tfidf = TfidfVectorizer(stop_words='english', max_features=500)
X = tfidf.fit_transform(df_cleaned['Cleaned_Description'])

# KMeans kümeleme
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=30, random_state=42)
df_cleaned['Category'] = kmeans.fit_predict(X)

# Kümeler için en önemli kelimeleri alalım
terms = tfidf.get_feature_names_out()

# Kümeler için en sık geçen kelimeler
for i in range(30):  # 10 kategori olduğu varsayılıyor
    print(f"Kategori {i}:")
    cluster_terms = kmeans.cluster_centers_[i].argsort()[-3:][::-1]  # 5 en yüksek ağırlıklı kelime
    print([terms[idx] for idx in cluster_terms])


Kategori 0:
['skull', 'black', 'bag']
Kategori 1:
['notebook', 'babushka', 'set']
Kategori 2:
['card', 'wallet', 'travel']
Kategori 3:
['set', 'tin', 'pantry']
Kategori 4:
['bag', 'lunch', 'design']
Kategori 5:
['box', 'lunch', 'trinket']
Kategori 6:
['pink', 'heart', 'polkadot']
Kategori 7:
['birthday', 'card', 'happy']
Kategori 8:
['assorted', 'colour', 'ornament']
Kategori 9:
['blue', 'mug', 'wrap']
Kategori 10:
['holder', 'tlight', 'hanging']
Kategori 11:
['paper', 'kit', 'chain']
Kategori 12:
['bag', 'jumbo', 'retrospot']
Kategori 13:
['wall', 'diner', 'art']
Kategori 14:
['bottle', 'hot', 'water']
Kategori 15:
['pack', 'tissue', 'napkin']
Kategori 16:
['cake', 'christmas', 'case']
Kategori 17:
['doormat', 'home', 'sweet']
Kategori 18:
['sign', 'metal', 'french']
Kategori 19:
['hand', 'warmer', 'design']
Kategori 20:
['tea', 'towel', 'set']
Kategori 21:
['bunting', 'party', 'spotty']
Kategori 22:
['candle', 'set', 'party']
Kategori 23:
['wooden', 'frame', 'antique']
Kategori 24:
[

C:\Users\EXCALIBUR\AppData\Local\Temp\ipykernel_17000\2449455936.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Category'] = kmeans.fit_predict(X)


In [ ]:
category_names = {
    0: "Bags",
    1: "Notebooks & Office Supplies",
    2: "Wallets & Travel Accessories",
    3: "Kitchen & Storage Containers",
    4: "Lunch Bags & Designs",
    5: "Storage Boxes & Accessories",
    6: "Decorative Items",
    7: "Birthday & Celebration Cards",
    8: "Decorative Ornaments & Accessories",
    9: "Mugs & Cups",
    10: "Candle & Decorative Accessories",
    11: "Paper Products & DIY Craft Kits",
    12: "Shopping Bags",
    13: "Wall Decorations & Paintings",
    14: "Thermos & Water Bottles",
    15: "Napkin & Tissue Sets",
    16: "Cake & Dessert Accessories",
    17: "Door Mats & Home Decorations",
    18: "Metal Signs & Accessories",
    19: "Hand Warmers & Winter Products",
    20: "Kitchen Towels & Sets",
    21: "Party Supplies",
    22: "Candle & Decorative Accessories",
    23: "Wooden Frames & Decorations",
    24: "Clocks & Alarm Clocks",
    25: "Children’s Products",
    26: "Wicker Decorative Items",
    27: "Hanging Decorative Items",
    28: "Jam & Storage Jars",
    29: "Cookie Cutters & Sets"
}

df_cleaned["Category_Name"] = df_cleaned["Category"].map(category_names)


C:\Users\EXCALIBUR\AppData\Local\Temp\ipykernel_17000\1713574557.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["Category_Name"] = df_cleaned["Category"].map(category_names)


In [ ]:
df_cleaned.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales,Cleaned_Description,Vector,Category,Category_Name
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,white hanging heart tlight holder,"[0.9714672, -0.26128998, -0.70263577, 1.964754...",10,Candle & Decorative Accessories
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,white metal lantern,"[1.099966, -0.25040722, -0.76072717, -0.313656...",9,Mugs & Cups
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,cream cupid heart coat hanger,"[1.3082095, -0.2327377, 0.17611119, 0.48868498...",6,Decorative Items
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,knitted union flag hot water bottle,"[2.0762331, 1.2777542, -2.1614716, -0.78004295...",14,Thermos & Water Bottles
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,red woolly hottie white heart,"[1.0413097, 0.5996016, -0.79946333, 1.1287888,...",6,Decorative Items


In [ ]:
df_cleaned = df_cleaned.drop(columns=["Vector", "Category","Cleaned_Description"])


In [ ]:
df_cleaned.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales,Category_Name
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,Candle & Decorative Accessories
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,Mugs & Cups
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,Decorative Items
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,Thermos & Water Bottles
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,Decorative Items


In [ ]:
df_cleaned.to_csv('data_with_categories.csv', index=False)


In [ ]:
data = pd.read_csv("data_with_categories.csv")

In [ ]:
# En çok satış yaplan kategoriler

best_cat = data.groupby('Category_Name').agg({'Sales': 'sum'}).sort_values(by='Sales', ascending=False)
print(best_cat)

                                          Sales
Category_Name                                  
Mugs & Cups                         2237948.654
Shopping Bags                        708128.590
Kitchen & Storage Containers         619313.960
Candle & Decorative Accessories      427559.270
Paper Products & DIY Craft Kits      409608.540
Storage Boxes & Accessories          401557.680
Wooden Frames & Decorations          400560.550
Decorative Items                     387776.890
Cake & Dessert Accessories           365582.400
Door Mats & Home Decorations         352456.550
Lunch Bags & Designs                 230132.040
Metal Signs & Accessories            228201.530
Party Supplies                       208183.540
Wall Decorations & Paintings         200903.240
Thermos & Water Bottles              195639.230
Jam & Storage Jars                   184980.960
Decorative Ornaments & Accessories   173256.070
Wicker Decorative Items              165085.710
Kitchen Towels & Sets                146

In [ ]:
# En çok stoğu bulunan kategroiler

quant_cat=data.groupby('Category_Name').agg({'Quantity':'sum'}).sort_values(by='Quantity',ascending=False)
print(quant_cat)

                                    Quantity
Category_Name                               
Mugs & Cups                          1259369
Shopping Bags                         390930
Cake & Dessert Accessories            357672
Candle & Decorative Accessories       315336
Kitchen & Storage Containers          259214
Decorative Items                      256472
Paper Products & DIY Craft Kits       236541
Storage Boxes & Accessories           210066
Decorative Ornaments & Accessories    185295
Wooden Frames & Decorations           173449
Lunch Bags & Designs                  162620
Napkin & Tissue Sets                  155657
Wallets & Travel Accessories          151504
Hanging Decorative Items              144285
Jam & Storage Jars                    133487
Metal Signs & Accessories             130352
Bags                                   76887
Kitchen Towels & Sets                  73107
Door Mats & Home Decorations           66363
Birthday & Celebration Cards           65662
Hand Warme

In [ ]:
data["Total_Revenue"] = data["Quantity"] * data["UnitPrice"]
category_revenue = data.groupby("Category_Name")["Total_Revenue"].sum().sort_values(ascending=False)
category_revenue.head(10)  # En çok gelir getiren 10 kategori


Category_Name
Mugs & Cups                        2237948.654
Shopping Bags                       708128.590
Kitchen & Storage Containers        619313.960
Candle & Decorative Accessories     427559.270
Paper Products & DIY Craft Kits     409608.540
Storage Boxes & Accessories         401557.680
Wooden Frames & Decorations         400560.550
Decorative Items                    387776.890
Cake & Dessert Accessories          365582.400
Door Mats & Home Decorations        352456.550
Name: Total_Revenue, dtype: float64

In [ ]:
from IPython.display import display

data["InvoiceDate"] = pd.to_datetime(data["InvoiceDate"])
data["Month"] = data["InvoiceDate"].dt.month
seasonal_sales = data.groupby(["Month", "Category_Name"])["Quantity"].sum().unstack()

display(seasonal_sales)  # Jupyter'de düzgün gösterir


Category_Name,Bags,Birthday & Celebration Cards,Cake & Dessert Accessories,Candle & Decorative Accessories,Children’s Products,Clocks & Alarm Clocks,Cookie Cutters & Sets,Decorative Items,Decorative Ornaments & Accessories,Door Mats & Home Decorations,...,Notebooks & Office Supplies,Paper Products & DIY Craft Kits,Party Supplies,Shopping Bags,Storage Boxes & Accessories,Thermos & Water Bottles,Wall Decorations & Paintings,Wallets & Travel Accessories,Wicker Decorative Items,Wooden Frames & Decorations
Month,,,,,,,,,,,,,,,,,,,,,
1,4458,5122,12672,19638,2987,914,3431,19138,11683,3367,...,2214,9365,2616,20447,13937,1551,1233,4325,5231,5591
2,4402,4838,9625,16960,1974,1137,2147,15984,8991,2371,...,1229,8560,3259,19081,10944,620,929,7668,2517,6960
3,6751,5187,17346,18676,2951,980,2867,18681,12204,3205,...,1360,12502,4425,33858,17037,408,837,8431,1950,9129
4,4321,4493,13742,20901,2613,2640,1287,15537,13162,2638,...,1863,9841,5110,18154,9212,312,1159,10484,906,5911
5,5230,6087,14862,27186,3198,896,1495,20093,14660,5499,...,1906,9934,8456,35858,14897,551,1828,9826,4792,10107
6,7505,5203,15276,21790,3656,1620,1013,19771,13970,9333,...,2886,9380,4997,33688,10671,401,2409,9434,2988,11483
7,5827,4608,29029,17930,4233,1190,1615,19976,16097,6068,...,2119,6717,5303,28287,11648,699,2936,10435,2597,12328
8,7380,5421,28791,23553,4606,3001,1486,22293,15319,2531,...,2288,7030,4171,42838,18280,2350,2571,10193,4258,13265
9,8524,5741,56975,28037,5284,2486,1966,25489,16163,14293,...,2159,14416,3314,40923,22177,8328,2959,17239,8286,20095


In [ ]:
top_customers = data.groupby("CustomerID")["Quantity"].sum().nlargest(10).index
df_top_customers = data[data["CustomerID"].isin(top_customers)]
df_top_customers.groupby("Category_Name")["Quantity"].sum().sort_values(ascending=False)


Category_Name
Mugs & Cups                           161903
Paper Products & DIY Craft Kits        98779
Jam & Storage Jars                     80497
Shopping Bags                          50587
Cake & Dessert Accessories             43232
Kitchen & Storage Containers           41738
Storage Boxes & Accessories            37418
Decorative Items                       36473
Candle & Decorative Accessories        36423
Lunch Bags & Designs                   26558
Door Mats & Home Decorations           22575
Wallets & Travel Accessories           21901
Metal Signs & Accessories              20512
Hanging Decorative Items               19492
Decorative Ornaments & Accessories     17222
Wooden Frames & Decorations            16169
Wall Decorations & Paintings           15770
Napkin & Tissue Sets                   14471
Wicker Decorative Items                12660
Birthday & Celebration Cards            9672
Children’s Products                     9129
Bags                                    8